<a href="https://colab.research.google.com/github/vedica1011/NLP_BERT_TL/blob/main/FlanT_5_sentiment_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.6.2 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 2.2 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [37]:
%%time
model_name='google/flan-t5-small'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name,truncation=True,padding=True)

CPU times: user 4.06 s, sys: 375 ms, total: 4.44 s
Wall time: 8 s


In [7]:
pwd

'/content'

In [8]:
import pandas as pd
data = pd.read_csv("twitter_validation.csv",header=None,usecols=[2,3])
data.columns = ['labels','text']

In [9]:
data.head()

,labels,text
0,Irrelevant,I mentioned on Facebook that I was struggling ...
1,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,Negative,@Microsoft Why do I pay for WORD when it funct...
3,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,Neutral,Now the President is slapping Americans in the...


In [10]:
data.iloc[100,:]

labels                                              Neutral
text      🤔 Not sure where all your data is going? Frust...
Name: 100, dtype: object

In [11]:
data.shape

(1000, 2)

In [12]:
all_labels = data['labels'].unique()
all_labels

array(['Irrelevant', 'Neutral', 'Negative', 'Positive'], dtype=object)

In [13]:
data.to_csv('text_classs.csv',index=False)


In [14]:
#train_data = load_dataset('text_classs.csv',file)
train_data = load_dataset('csv', data_files='text_classs.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-99908963c9dfc48c/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
train_data

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 1000
    })
})

In [16]:
s = " "
for i in all_labels:
  s = s+i+" ,"
print(s)

 Irrelevant ,Neutral ,Negative ,Positive ,


In [17]:
index = 100
question = "Extract the most probable labels from the given sentence :"+train_data['train'][index]['text'] + "from these labels:"+s
answers = train_data['train'][index]['labels']
prompt = f"""
Answer the following question.

{question}

Answer:
"""
inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'Answer:\n{answers}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Answer the following question.

Extract the most probable labels from the given sentence :🤔 Not sure where all your data is going? Frustrated by your kids using it all immediately? 😠 With the My Verizon app, the power is in your hands! ⚡ Download it now in the App store and via Google Play. 👍from these labels: Irrelevant ,Neutral ,Negative ,Positive ,

Answer:

---------------------------------------------------------------------------------------------------
Answer:
Neutral

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Positive, Negative, Positive


In [18]:
def get_prompt(text,labels):
  prompt = f"""Answer the following question. \n Extract the most probable labels from the given sentence : {text}.
  from these labels: {labels}."""
  return prompt

In [19]:
get_prompt(train_data['train']['text'][0],s)

'Answer the following question. \n Extract the most probable labels from the given sentence : I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣.\n  from these labels:  Irrelevant ,Neutral ,Negative ,Positive ,.'

In [20]:
import re

def data_preprocess(examples):
  examples['text'] = get_prompt(examples['text'],s)
  return examples

In [21]:
final_data = train_data['train'].map(data_preprocess)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [22]:
final_data['text'][0]

'Answer the following question. \n Extract the most probable labels from the given sentence : I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣.\n  from these labels:  Irrelevant ,Neutral ,Negative ,Positive ,.'

In [146]:
final_data['text'][100]

'Answer the following question. \n Extract the most probable labels from the given sentence : 🤔 Not sure where all your data is going? Frustrated by your kids using it all immediately? 😠 With the My Verizon app, the power is in your hands! ⚡ Download it now in the App store and via Google Play. 👍.\n  from these labels:  Irrelevant ,Neutral ,Negative ,Positive ,.'

In [160]:
final_data['labels'][10]

'Positive'

In [42]:
def general_tokenizer(examples):
  return tokenizer(examples["text"],text_target=examples["labels"],max_length=1024,truncation=True)

In [43]:
custom_token_data = final_data.map(general_tokenizer,batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [44]:
print(custom_token_data[0])

{'labels': [27, 52, 30374, 1], 'text': 'Answer the following question. \n Extract the most probable labels from the given sentence : I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣.\n  from these labels:  Irrelevant ,Neutral ,Negative ,Positive ,.', 'input_ids': [11801, 8, 826, 822, 5, 18742, 8, 167, 23093, 11241, 45, 8, 787, 7142, 3, 10, 27, 2799, 30, 1376, 24, 27, 47, 8335, 21, 6620, 12, 281, 21, 3, 9, 661, 8, 119, 239, 6, 84, 65, 118, 15459, 57, 3059, 22, 7, 248, 22708, 23, 15, 38, 458, 566, 9, 63, 1306, 54, 22, 17, 129, 91, 13, 1953, 22, 11, 1219, 12, 112, 1907, 51, 9, 6, 113, 230, 317, 7, 27, 22, 51, 3, 9, 19743, 6, 9412, 568, 3, 2, 5, 45, 175, 11241, 10, 27, 52, 30374, 3, 6, 26288, 8792, 3, 6, 567, 15, 122, 1528, 3, 6, 345, 32, 7, 23, 3268, 3, 6, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1

In [45]:
from transformers import DataCollatorForSeq2Seq,Seq2SeqTrainingArguments,Trainer
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=original_model)

In [81]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./model",
    logging_steps = 10,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=1,
    save_steps=12,
    save_total_limit=2)

In [89]:
!pip install git+https://github.com/huggingface/peft

  Cloning https://github.com/huggingface/peft to /tmp/pip-req-build-r30unq7e
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-r30unq7e
  Resolved https://github.com/huggingface/peft to commit cf04d0353f0343cbf66627228c4495f51669af34
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.7.2.dev0-py3-none-any.whl size=169456 sha256=4b580ac4cbae9271b2cc382ad0619578b315e926d5d78ab497da7f5c93c7319d
  Stored in directory: /tmp/pip-ephem-wheel-cache-d2rwnqqo/wheels/4c/16/67/1002a2d4daa822eff130e6d85b90051b75d2ce0d26b9448e4a
Successfully built peft
  Attempting uninstall: peft
    Found existing installation: peft 0.6.2
    Uninstalling peft-0.6.2:
      Successfully uninstalled peft-0.6.2


In [29]:
from peft import LoraConfig,get_peft_config,get_peft_model,get_peft_model_state_dict,prepare_model_for_kbit_training,AutoPeftModelForSeq2SeqLM,TaskType

In [30]:
import peft
peft.__version__

'0.7.2.dev0'

In [47]:
original_model.gradient_checkpointing_enable()
original_model = prepare_model_for_kbit_training(original_model)

In [82]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules = ['q','v'],
    task_type = TaskType.SEQ_2_SEQ_LM,
    bias = 'none'
)
original_model = get_peft_model(original_model,peft_config)

In [83]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"\ntrainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print_number_of_trainable_model_parameters(original_model)

'\ntrainable model parameters: 1376256\nall model parameters: 78337408\npercentage of trainable model parameters: 1.76%'

In [84]:
from transformers import Seq2SeqTrainer

In [85]:
trainer = Seq2SeqTrainer(
    model = original_model,
    args = training_args,
    train_dataset = custom_token_data,
    tokenizer = tokenizer,
    data_collator = data_collator
)

In [86]:
%%time
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
2,1.212800
4,1.225900
6,1.222800
8,1.261700
10,0.896500
12,1.129100
14,1.064000
16,1.168000
18,1.156200
20,1.022700


CPU times: user 8min 50s, sys: 9.79 s, total: 9min
Wall time: 9min 9s


TrainOutput(global_step=63, training_loss=1.1100813755913386, metrics={'train_runtime': 549.1879, 'train_samples_per_second': 1.821, 'train_steps_per_second': 0.115, 'total_flos': 52538021412864.0, 'train_loss': 1.1100813755913386, 'epoch': 1.0})

In [110]:
#trainer.save_model('new_model1')
trainer.model.save_pretrained("new_model")
trainer.tokenizer.save_pretrained("new_model")

('new_model/tokenizer_config.json',
 'new_model/special_tokens_map.json',
 'new_model/tokenizer.json')

In [107]:
from peft import PeftModel

In [137]:
peft_model = PeftModel.from_pretrained(original_model,"/content/new_model")
tokenizer1 = AutoTokenizer.from_pretrained("/content/new_model")
peft_model.to('cpu')

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): PeftModelForSeq2SeqLM(
      (base_model): LoraModel(
        (model): PeftModelForSeq2SeqLM(
          (base_model): LoraModel(
            (model): T5ForConditionalGeneration(
              (shared): Embedding(32128, 512)
              (encoder): T5Stack(
                (embed_tokens): Embedding(32128, 512)
                (block): ModuleList(
                  (0): T5Block(
                    (layer): ModuleList(
                      (0): T5LayerSelfAttention(
                        (SelfAttention): T5Attention(
                          (q): lora.Linear(
                            (base_layer): Linear(in_features=512, out_features=384, bias=False)
                            (lora_dropout): ModuleDict(
                              (default): Identity()
                            )
                            (lora_A): ModuleDict(
                              (default): Linear(in_features=512, out_features=32, bi

In [135]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer


In [161]:
index = 10
question = "Extract the most probable labels from the given sentence :"+train_data['train'][index]['text'] + "from these labels:"+s
answers = train_data['train'][index]['labels']
prompt = f"""
Answer the following question.

{question}

Answer:
"""
inputs = tokenizer1(prompt, return_tensors='pt').input_ids
original_model_outputs = peft_model.generate(input_ids=inputs,
                                             generation_config=GenerationConfig(max_new_tokens=200))

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'Answer:\n{answers}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Answer the following question.

Extract the most probable labels from the given sentence :The professional dota 2 scene is fucking exploding and I completely welcome it.

Get the garbage out.from these labels: Irrelevant ,Neutral ,Negative ,Positive ,

Answer:

---------------------------------------------------------------------------------------------------
Answer:
Positive

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Positive, Negative, Positive


In [162]:
tokenizer1.decode(original_model_outputs[0])

'<pad> Negative</s>'